**What is the Math behind COVID-19?**
The curve used to model this situation is called a logistic curve, an S-shaped curve that describes population growth of both viruses and people, as well as other phenomena in economics and science.
![curve](https://www.nctm.org/uploadedImages/Content/General/pandemics/Pandemic_curve.jpg)
We will fit this simple curve to our virus data (as provided by Kaggle) and see how well it performs on the leaderboard.

In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
import scipy.optimize as opt
import os

In [ ]:
train = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-2/train.csv")
test = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-2/test.csv")
train.head()

Filter out the 0's, countries are affected differently chronologically based on geography, holidaymakers, season etc... I could be wrong though, I mostly have online gambling experience, no epidemiological experience lol!
edit: added them back because that produces a better score. Guess thats empirical testing for you.

In [ ]:
train_ = train[train["ConfirmedCases"] >= 0]
train_.head()

Let us replace the empty Province_State cases with the Country_Region:

In [ ]:
EMPTY_VAL = "EMPTY_VAL"

def fillState(state, country):
    if state == EMPTY_VAL: return country
    return state

train_['Province_State'].fillna(EMPTY_VAL, inplace=True)
train_['Province_State'] = train_.loc[:, ['Province_State', 'Country_Region']].apply(lambda x : fillState(x['Province_State'], x['Country_Region']), axis=1)
test['Province_State'].fillna(EMPTY_VAL, inplace=True)
test['Province_State'] = test.loc[:, ['Province_State', 'Country_Region']].apply(lambda x : fillState(x['Province_State'], x['Country_Region']), axis=1)
test.head()

The logistic curve 𝑓(𝑥)=𝐿/(1+𝑒−𝑘(𝑥−𝑥0)) is what we will use.

We will also use another variation of the function: 𝑓𝐿,b,𝑘,𝑥0(𝑥) = 𝐿/(1+𝑒−𝑘(𝑥−𝑥0))+b, depending on which works for the data.

Some more commentary on scipy:

In SciPy, nonlinear least squares curve fitting works by minimizing the following cost function:

S(β)=∑i=1n(yi−fβ(xi))2

Here, β is the vector of parameters (in our example, β=(L,b,k,x0).

Nonlinear least squares is really similar to linear least squares for linear regression. Whereas the function f is linear in the parameters with the linear least squares method, it is not linear here. Therefore, the minimization of S(β) cannot be done analytically by solving the derivative of S with respect to β. SciPy implements an iterative method called the Levenberg-Marquardt algorithm (an extension of the Gauss-Newton algorithm).

Let us test the functions out:

In [ ]:
train_['row_number'] = train_.groupby(['Country_Region', 'Province_State']).cumcount()
x = train_[train_["Country_Region"] == 'China'][train_["Province_State"] == 'Hubei']['row_number']
y = train_[train_["Country_Region"] == 'China'][train_["Province_State"] == 'Hubei']['ConfirmedCases']
y_ = train_[train_["Country_Region"] == 'China'][train_["Province_State"] == 'Hubei']['Fatalities']

def f(x, L, b, k, x_0):
    return L / (1. + np.exp(-k * (x - x_0))) + b


def logistic(xs, L, k, x_0):
    result = []
    for x in xs:
        xp = k*(x-x_0)
        if xp >= 0:
            result.append(L / ( 1. + np.exp(-xp) ) )
        else:
            result.append(L * np.exp(xp) / ( 1. + np.exp(xp) ) )
    return result

p0 = [max(y), 0.0,max(x)]
p0_ = [max(y_), 0.0,max(x)]
x_ = np.arange(0, 100, 1).tolist()
try:
    popt, pcov = opt.curve_fit(logistic, x, y,p0)
    yfit = logistic(x_, *popt)
    popt_, pcov_ = opt.curve_fit(logistic, x, y_,p0_)
    yfit_ = logistic(x_, *popt_)
except:
    popt, pcov = opt.curve_fit(f, x, y, method="lm", maxfev=5000)
    yfit = f(x_, *popt)
    popt_, pcov_ = opt.curve_fit(f, x, y_, method="lm", maxfev=5000)
    yfit_ = f(x_, *popt_)
    #print("problem")


fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.plot(x, y, 'o', label ='Actual Cases')
ax.plot(x_, yfit, '-', label ='Fitted Cases')

ax.plot(x, y_, 'o', label ='Actual Fatalities')
ax.plot(x_, yfit_, '-', label ='Fitted fatalities')
ax.title.set_text('China - Hubei province')
plt.legend(loc="center right")
plt.show()

Seems reasonable to me anyway. Go ahead and test a few. There are one or two that come back erroneous but for the most part its pretty good at it.
Now lets create a dimensional lookup(LUP) of the composite (Country_Region,Province_State):

In [ ]:
unique = pd.DataFrame(train_.groupby(['Country_Region', 'Province_State'],as_index=False).count())
unique.head()

Now we are ready to iterate through the trianing data and fit curves on the data:

In [ ]:
import datetime as dt

def date_day_diff(d1, d2):
    delta = dt.datetime.strptime(d1, "%Y-%m-%d") - dt.datetime.strptime(d2, "%Y-%m-%d")
    return delta.days

log_regions = []

for index, region in unique.iterrows():
    st = region['Province_State']
    co = region['Country_Region']
    
    rdata = train_[(train_['Province_State']==st) & (train_['Country_Region']==co)]

    t = rdata['Date'].values
    t = [float(date_day_diff(d, t[0])) for d in t]
    y = rdata['ConfirmedCases'].values
    y_ = rdata['Fatalities'].values

    p0 = [max(y), 0.0, max(t)]
    p0_ = [max(y_), 0.0, max(t)]
    try:
        popt, pcov = opt.curve_fit(logistic, t, y, p0, maxfev=10000)
        try:
            popt_, pcov_ = opt.curve_fit(logistic, t, y_, p0_, maxfev=10000)
        except:
            popt_, pcov_ = opt.curve_fit(f, t, y_,method="trf", maxfev=10000)
        log_regions.append((co,st,popt,popt_))
    except:
        popt, pcov = opt.curve_fit(f, t, y,method="trf", maxfev=10000)
        popt_, pcov_ = opt.curve_fit(f, t, y_,method="trf", maxfev=10000)
        log_regions.append((co,st,popt,popt_))

print("All done!")

Convert it to a dataframe:

In [ ]:
log_regions = pd.DataFrame(log_regions)
log_regions.head()

Rename columns:

In [ ]:
log_regions.columns = ['Country_Region','Province_State','ConfirmedCases','Fatalities']
log_regions.head(1)

Wondering if i should curbe outliers here:

In [ ]:
data = log_regions['ConfirmedCases'].str[1]
bins = np.arange(0, 10, 0.01)
plt.hist(data,bins=bins, alpha=0.5)
plt.xlim([0,1])
plt.ylabel('count')
plt.show()
log_regions['ConfirmedCases'].str[1].quantile([.1, .25, .5, .75, .95, .99])

In [ ]:
#log_regions.loc[log_regions['ConfirmedCases'].str[1] > 1.0, 'ConfirmedCases'] = 0.267766

In [ ]:
log_regions['ConfirmedCases'].str[1].quantile([.1, .25, .5, .75, .95, .99])

Let us test graphing a curve from the fitted parameters:

In [ ]:
T = np.arange(0, 100, 1).tolist()
popt = list(log_regions[log_regions["Country_Region"] == 'Italy'][log_regions["Province_State"] == 'Italy']['ConfirmedCases'])[0]
popt_ = list(log_regions[log_regions["Country_Region"] == 'Italy'][log_regions["Province_State"] == 'Italy']['Fatalities'])[0]

try:
    yfit = logistic(T, *popt)
    yfit_ = logistic(T, *popt_)
except:
    yfit = f(T, *popt)
    yfit_ = f(T, *popt_)
    

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.plot(T, yfit, label="Fitted ConfirmedCases")
ax.plot(T, yfit_, label="Fitted Fatalities")
ax.title.set_text('Italy fitted params')
plt.legend(loc="upper left")
plt.show()

Now we iterate our fits through the test data for submission:

In [ ]:
for index, rt in log_regions.iterrows():
    st = rt['Province_State']
    co = rt['Country_Region']
    popt = list(['ConfirmedCases'])
    popt_ = list(rt['Fatalities'])
    print(co,st,popt,popt_)

Lets try replace parameters for the locations where there havent been any fatalities so far, leaning on the notion that fatalities follow cases according to observation.
what we will do is work out the median ratios of the parameters and apply them to the confirmed cases parameters.

In [ ]:
data0 = log_regions['Fatalities'].str[0]/log_regions['ConfirmedCases'].str[0]
data1 = log_regions['Fatalities'].str[1]/log_regions['ConfirmedCases'].str[1]
data2 = log_regions['Fatalities'].str[2]/log_regions['ConfirmedCases'].str[2]
bins = np.arange(0, 3, 0.01)
plt.hist(data,bins=bins, alpha=0.5)
plt.xlim([0,3])
plt.ylabel('count')
plt.ylim([0,5])
plt.show()
fp = np.array([data0.median(),data1.median(),data2.median()])
fp

see it in action first:

In [ ]:
for index, rt in log_regions.iterrows():
    st = rt['Province_State']
    co = rt['Country_Region']
    popt = list(rt['ConfirmedCases'])
    popt_ = list(rt['Fatalities'])
    
    if popt_ == [0.0,0.0,69.0]:
        popt_ = np.multiply(fp,popt)
        print(co,st,popt,popt_)

In [ ]:
submission = []

for index, rt in log_regions.iterrows():
    st = rt['Province_State']
    co = rt['Country_Region']
    popt = list(rt['ConfirmedCases'])
    popt_ = list(rt['Fatalities'])
    if popt_ == [0.0,0.0,69.0]:
        popt_ = np.multiply(fp,popt)
    print(co,st,popt,popt_)
    rtest = test[(test['Province_State']==st) & (test['Country_Region']==co)]
    for index, rt in rtest.iterrows():
        try:
            tdate = rt['Date']
            ca = logistic([date_day_diff(tdate, min(train_[(train_['Province_State']==st) & (train_['Country_Region']==co)]['Date'].values))], *popt)
            try:
                fa = logistic([date_day_diff(tdate, min(train_[(train_['Province_State']==st) & (train_['Country_Region']==co)]['Date'].values))], *popt_)
            except:
                fa = f([date_day_diff(tdate, min(train_[(train_['Province_State']==st) & (train_['Country_Region']==co)]['Date'].values))], *popt_)
            submission.append((rt['ForecastId'], int(ca[0]), int(fa[0])))
        except:
            tdate = rt['Date']
            ca = f([date_day_diff(tdate, min(train_[(train_['Province_State']==st) & (train_['Country_Region']==co)]['Date'].values))], *popt)
            fa = f([date_day_diff(tdate, min(train_[(train_['Province_State']==st) & (train_['Country_Region']==co)]['Date'].values))], *popt_)
            submission.append((rt['ForecastId'], int(ca[0]), int(fa[0])))

print("All done!") 

Prepare results for submission:

In [ ]:
submission = pd.DataFrame(submission)
submission.columns = ['ForecastId','ConfirmedCases','Fatalities']
submission.to_csv('./submission.csv', index = False)
print("submission ready!")

Public Score
0.19234

Rank 66/220 as at 12:45 SAST Sunday, 29 March 2020.

Not too bad for a simple approach. Let me know if you have any suggestions and how you fare!

T